In [15]:
# ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# エンコーディング
from sklearn.preprocessing import OrdinalEncoder

# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [3]:
# データ準備
df = sns.load_dataset('titanic')

# 単純に欠損値のあるデータを消す
df.dropna(inplace=True)
# X, yを作成 loc[行:列]
X = df.loc[:, (df.columns!='survived') & (df.columns!= 'alive')]
y = df['survived']
# ラベルエンコーディング
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
oe.set_output(transform='pandas')  # デフォルトだとnumpy配列で出力されるから、dfの形に変える。
X = oe.fit_transform(X)
# 学習データを分ける。
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


## Stacking(スクラっチ)

In [19]:
class StacingClassifierCV():

    def __init__(self, estimators,final_estimator, cv):
        self.estimators = estimators
        self.final_estimator = final_estimator
        self.cv = cv

    def fit(self, X, y):
        pred_features = {}
        # 1層目のモデル学習
        for model_name, model in self.estimators:
            preds = []
            new_y = []

            for train_idx, val_idx in self.cv.split(X):
                X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
                y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
                model.fit(X_train, y_train)
                pred = model.predict_proba(X_val)[:, 1].tolist()
                preds = preds + pred
                new_y = new_y + y_val
            pred_features[model_name] = preds

        # 2層目のモデル学習
        return pred_features
        # new_X = pd.DataFrame(pred_features)
        
            


In [20]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
final_estimator = RandomForestClassifier()
my_stacking = StacingClassifierCV(estimators=[('rf', RandomForestClassifier()), ('knn', KNeighborsClassifier())], final_estimator=final_estimator, cv=cv)

In [22]:
# my_stacking.fit(X_train, y_train)

In [23]:
# X_train.iloc[train_idx]